# Revising manuscript. 
## Data preparing according to reviewer's comments.

In [161]:
import pandas as pd

In [162]:
cre = pd.read_csv('ephys_data_cre.csv')
# as review 2 minor commemts # 11 . technical values removed.
cre = cre.drop(['reporter_status', 'height', 'input_resistance_mohm', 'vm_for_sag', 'slow_trough_t_short_square', 'slow_trough_v_short_square'], axis=1)
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 40 columns):
adaptation                                816 non-null float64
avg_isi                                   889 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_square                        969 non-null float64
peak_v_ramp  

In [163]:
cre_dropna = cre.dropna(axis=0, how='any')

In [164]:
a = cre['transgenic_line'].value_counts()
b= cre_dropna['transgenic_line'].value_counts()
a - b

Chat-IRES-Cre-neo    11
Chrna2-Cre_OE25       1
Ctgf-T2A-dgCre        0
Cux2-CreERT2         13
Gad2-IRES-Cre         4
Htr3a-Cre_NO152      16
Ndnf-IRES2-dgCre      1
Nkx2-1-CreERT2        1
Nr5a1-Cre             9
Ntsr1-Cre_GN220      15
Pvalb-IRES-Cre       38
Rbp4-Cre_KL100       13
Rorb-IRES2-Cre       14
Scnn1a-Tg2-Cre        3
Scnn1a-Tg3-Cre        7
Slc17a6-IRES-Cre      1
Sst-IRES-Cre         20
Vip-IRES-Cre          1
Name: transgenic_line, dtype: int64

In [165]:
(a-b)/a

Chat-IRES-Cre-neo    0.550000
Chrna2-Cre_OE25      0.032258
Ctgf-T2A-dgCre       0.000000
Cux2-CreERT2         0.178082
Gad2-IRES-Cre        0.363636
Htr3a-Cre_NO152      0.179775
Ndnf-IRES2-dgCre     0.066667
Nkx2-1-CreERT2       0.043478
Nr5a1-Cre            0.130435
Ntsr1-Cre_GN220      0.312500
Pvalb-IRES-Cre       0.267606
Rbp4-Cre_KL100       0.180556
Rorb-IRES2-Cre       0.116667
Scnn1a-Tg2-Cre       0.083333
Scnn1a-Tg3-Cre       0.116667
Slc17a6-IRES-Cre     0.333333
Sst-IRES-Cre         0.180180
Vip-IRES-Cre         0.050000
Name: transgenic_line, dtype: float64

adaptation                                816 non-null float64


In [166]:
cre['peak_t_short_square']= (cre['peak_t_short_square']-1.02)*1000
cre['peak_t_short_square']
# make time unit as 3ms, short square pulse stimuli applied 3ms, so the start of stimuli set to zero.

0      3.300000
1      3.480000
2      3.475000
3      3.508333
4      3.560000
5      3.597000
6      3.541000
7      3.311000
8      3.390000
9      3.570000
10     3.478000
11     3.030625
12     3.508333
13     3.841667
14     3.140000
15     3.462000
16     3.427500
17     3.160000
18     3.318750
19     3.580000
20     3.336000
21     3.432000
22     3.256667
23     3.460000
24     3.657000
25     3.525000
26     3.320000
27     3.673333
28     3.672000
29     3.344000
         ...   
939    3.430000
940    3.392000
941    3.365000
942    3.735000
943    3.420000
944    3.373000
945    3.436250
946    3.405000
947    3.410000
948    3.280000
949    3.515000
950    3.385000
951    3.342500
952    3.215000
953    3.227000
954    3.710000
955    3.448750
956    3.315000
957    3.552000
958    3.524000
959    3.343750
960    3.292000
961    3.447500
962    3.303000
963    3.493750
964    3.413750
965    3.576000
966    3.410000
967    3.445000
968    3.592000
Name: peak_t_short_squar

In [167]:
cre['threshold_t_short_square'] = (cre['threshold_t_short_square']-1.02)*1000
cre['threshold_t_short_square']

0      2.913333
1      3.159000
2      3.001000
3      3.130000
4      3.120000
5      2.979000
6      3.197000
7      3.032000
8      3.010000
9      3.142000
10     2.943000
11     2.731875
12     3.150000
13     3.266667
14     2.917000
15     2.979000
16     3.043750
17     2.746667
18     3.002500
19     3.103000
20     2.946000
21     3.123000
22     2.856667
23     3.104000
24     3.137000
25     3.155000
26     2.970000
27     3.191667
28     3.168000
29     2.985000
         ...   
939    3.055000
940    3.055000
941    2.948333
942    3.289000
943    3.092500
944    3.033000
945    3.061250
946    3.046000
947    3.025000
948    2.970000
949    3.114000
950    3.055000
951    3.040000
952    2.895000
953    2.994000
954    3.190000
955    3.090000
956    3.053000
957    3.120000
958    3.144000
959    2.983750
960    3.008000
961    3.077500
962    3.033000
963    3.192500
964    3.025000
965    3.132000
966    3.095000
967    3.065000
968    3.164000
Name: threshold_t_short_

In [168]:
cre['fast_trough_t_short_square'] = (cre['fast_trough_t_short_square']-1.02)*1000
cre['fast_trough_t_short_square']

0      4.873333
1      4.593000
2      7.302000
3      4.665833
4      5.426667
5      5.615000
6      4.461000
7      4.103000
8      4.870000
9      5.506000
10     5.854000
11     3.790625
12     4.195833
13     5.882500
14     3.625000
15     5.411000
16     4.420000
17     4.846667
18     4.180000
19     5.553000
20     5.011000
21     4.358000
22     4.471667
23     5.428000
24     7.009000
25     4.895000
26     4.625000
27     6.062500
28     5.740000
29     4.485000
         ...   
939    4.815000
940    4.841000
941    5.006667
942    6.266000
943    5.032500
944    4.921000
945    9.810000
946    6.005000
947    5.352500
948    4.520000
949    7.092000
950    4.130000
951    8.495000
952    4.122500
953    3.877000
954    6.587857
955    5.132500
956    3.969000
957    5.192000
958    5.352000
959    5.056250
960    4.348000
961    9.131250
962    3.975000
963    4.292500
964    5.281250
965    5.360000
966    4.603750
967    4.965000
968    5.576000
Name: fast_trough_t_shor

In [169]:
cre['trough_t_short_square'] = (cre['trough_t_short_square']-1.02)*1000
cre['trough_t_short_square']

0      324.913333
1      213.473000
2      671.744000
3      475.571667
4      110.500000
5      453.574000
6      527.561000
7      436.287000
8      489.350000
9      231.151000
10     557.570000
11     343.485625
12     377.621667
13      34.124167
14     673.986000
15     391.247000
16     291.541250
17     811.826667
18     230.333750
19     426.244000
20     592.528000
21     395.886000
22     810.231667
23     585.676000
24     505.379000
25     518.510000
26     835.710000
27     527.079167
28      97.364000
29     720.821000
          ...    
939    669.315000
940    510.804000
941    689.963333
942    338.333000
943    446.345000
944    510.155000
945     27.485000
946    129.956000
947     66.213333
948    585.140000
949    589.005000
950    227.725000
951    179.678750
952      4.257500
953    612.955000
954    618.815000
955    619.258750
956    561.903000
957    584.992000
958    124.844000
959    682.153750
960    458.228000
961    512.405000
962     68.214000
963     26

In [170]:
cre['threshold_t_long_square'] = (cre['threshold_t_long_square']-1.02)*1000
cre['peak_t_long_square'] = (cre['peak_t_long_square']-1.02)*1000
cre['fast_trough_t_long_square'] = (cre['fast_trough_t_long_square']-1.02)*1000
cre['trough_t_long_square'] = (cre['trough_t_long_square']-1.02)*1000
# make time unit as ms, long square pulse stimuli applied for 1 s from 100 ms of sweep
# the start of stimuli set to zero.

In [171]:
cre['threshold_t_ramp'] = cre['threshold_t_ramp'] - 1.02
cre['peak_t_ramp'] = cre['peak_t_ramp'] -1.02
cre['fast_trough_t_ramp'] = cre['fast_trough_t_ramp'] - 1.02
cre['trough_t_ramp'] = cre['trough_t_ramp'] - 1.02
# ramp stimuli applied from 1.0 s of sweep
# the start of stimuli set to zero.

In [172]:
cre['height_short'] = cre['peak_v_short_square'] -cre['fast_trough_v_short_square']
cre['height_long'] = cre['peak_v_long_square'] - cre['fast_trough_v_long_square']
cre['height_ramp'] = cre['peak_v_ramp'] - cre['fast_trough_v_ramp']

In [173]:
(cre['height_long'] - cre['height_short']).mean

<bound method Series.mean of 0       0.187496
1     -22.099999
2      -2.700003
3      -8.223962
4      -9.635417
5      -6.293752
6      -6.718746
7     -13.662500
8      -4.562498
9      -8.037503
10     -4.874999
11     -3.972647
12     -0.749999
13    -11.859377
14    -13.056254
15     -4.850003
16     -2.234374
17     -2.489583
18     -3.273437
19     -7.656248
20    -10.331248
21     -0.293748
22     -1.500002
23     -8.206251
24    -10.037502
25     -3.007812
26     -2.718750
27     -6.911457
28     -9.443750
29     -1.706247
         ...    
939    -6.218749
940    -6.118752
941     6.052085
942    -7.125002
943   -27.765625
944    -3.981248
945    -4.148438
946    -5.131252
947    -8.656249
948    -2.007816
949    -9.474995
950    -4.367189
951   -11.320311
952   -13.906254
953    -4.537498
954    -7.058038
955    -9.882810
956   -15.675001
957   -10.212505
958    -3.600000
959    -0.414067
960   -11.150001
961   -13.718753
962    -9.350000
963    -7.437502
964   -10.773438
96

In [174]:
(cre['height_long'] - cre['height_ramp']).mean

<bound method Series.mean of 0      -0.833333
1     -20.000001
2      -7.718753
3       3.843746
4      -1.875001
5     -10.916670
6       2.427087
7      -0.020831
8      -3.458332
9     -16.208337
10     -1.562497
11           NaN
12      2.197920
13     -9.645837
14     14.249996
15     -3.635420
16     -4.249997
17     -2.062499
18     -0.552080
19      3.520835
20      7.552085
21      2.875002
22     -0.812500
23     -4.624996
24     -5.104167
25     -0.937500
26     -0.187496
27           NaN
28     -3.979164
29      1.250004
         ...    
939    -1.197917
940     3.781252
941     3.552086
942    -4.250003
943   -15.895835
944    -6.458330
945   -10.729166
946    -2.364583
947    -3.291666
948     0.291660
949    -4.218744
950     4.999998
951    -7.124999
952   -15.687505
953     7.833336
954   -13.145835
955    -0.604164
956     8.656252
957    -4.197919
958    -2.447918
959     0.156244
960     4.250002
961   -10.531254
962    24.125000
963    13.953124
964   -11.302085
96

In [175]:
(cre['height_ramp'] - cre['height_short']).mean

<bound method Series.mean of 0       1.020830
1      -2.099998
2       5.018749
3     -12.067707
4      -7.760415
5       4.622918
6      -9.145833
7     -13.641668
8      -1.104166
9       8.170833
10     -3.312502
11           NaN
12     -2.947919
13     -2.213540
14    -27.306250
15     -1.214583
16      2.015624
17     -0.427085
18     -2.721357
19    -11.177083
20    -17.883333
21     -3.168750
22     -0.687502
23     -3.581255
24     -4.933336
25     -2.070312
26     -2.531254
27           NaN
28     -5.464586
29     -2.956251
         ...    
939    -5.020832
940    -9.900003
941     2.499999
942    -2.874999
943   -11.869790
944     2.477082
945     6.580729
946    -2.766668
947    -5.364583
948    -2.299477
949    -5.256251
950    -9.367187
951    -4.195312
952     1.781251
953   -12.370834
954     6.087796
955    -9.278646
956   -24.331252
957    -6.014586
958    -1.152082
959    -0.570312
960   -15.400003
961    -3.187499
962   -33.475000
963   -21.390626
964     0.528647
96

In [176]:
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 43 columns):
adaptation                                816 non-null float64
avg_isi                                   889 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_square                        969 non-null float64
peak_v_ramp  

In [177]:
cre.loc[cre['transgenic_line'] == 'Ndnf-IRES2-dgCre', ['binary_neuron']] = 'Inhibitory'
cre.loc[cre['transgenic_line'] == 'Ndnf-IRES2-dgCre']

,adaptation,avg_isi,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,latency,...,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vrest,transgenic_line,firing_rate,binary_neuron,height_short,height_long,height_ramp
70,-0.016836,51.317500,0.253219,510.66,3.336407,5.040000,-53.656254,-53.114587,-56.575002,0.14006,...,3.277697,3.293132,2.619381,-67.046272,Ndnf-IRES2-dgCre,19.486530,Inhibitory,86.106253,80.156256,79.562504
99,0.008434,36.099259,0.480000,27.64,6.622927,7.312000,-47.437504,-46.395836,-56.981253,0.01428,...,2.780688,3.089430,2.601046,-72.885735,Ndnf-IRES2-dgCre,27.701399,Inhibitory,84.543754,72.437504,69.708337
258,0.005303,52.404706,0.328125,812.88,7.139487,7.336667,-49.937500,-51.989586,-60.036462,0.08436,...,3.979260,4.137271,2.575332,-76.635284,Ndnf-IRES2-dgCre,19.082256,Inhibitory,89.848962,76.843752,80.812504
264,0.002679,27.175000,0.335919,91.92,6.348600,4.668000,-53.500004,-52.239586,-55.675002,0.00868,...,2.502756,2.628995,2.568368,-69.332764,Ndnf-IRES2-dgCre,36.798528,Inhibitory,88.375003,80.906256,83.468754
356,NaN,20.160000,0.665800,42.40,5.765400,4.126667,-60.875004,-52.406250,-61.963545,0.01342,...,1.480181,1.053327,1.613715,-62.698147,Ndnf-IRES2-dgCre,49.603175,Inhibitory,91.453130,87.656256,63.343751
456,0.006942,28.112353,0.303640,517.10,5.328080,6.160000,-53.437500,-50.052087,-55.975002,0.02890,...,2.980973,2.859881,2.546601,-72.557991,Ndnf-IRES2-dgCre,35.571551,Inhibitory,89.081253,79.062502,77.041672
460,-0.025917,79.514545,0.257085,839.68,8.395487,7.660000,-52.406250,-49.739585,-58.562504,0.10944,...,3.787364,3.863806,2.875604,-75.740387,Ndnf-IRES2-dgCre,12.576315,Inhibitory,87.531256,66.281251,67.614587
525,-0.000414,31.932143,0.391826,947.66,5.427647,5.716667,-50.656250,-51.541669,-55.088544,0.09760,...,3.023153,3.061118,2.408996,-65.245323,Ndnf-IRES2-dgCre,31.316408,Inhibitory,78.812503,68.656252,69.645837
574,0.002607,31.416129,0.340848,77.90,5.234480,5.248000,-48.968750,-49.156253,-51.343752,0.01168,...,2.717109,2.526602,2.358213,-71.806557,Ndnf-IRES2-dgCre,31.830783,Inhibitory,78.300003,70.468750,73.104169
704,-0.007651,37.511200,0.312106,453.92,7.494600,8.415000,-52.656254,-49.458336,-60.109377,0.03728,...,3.007957,3.439152,2.371974,-75.434464,Ndnf-IRES2-dgCre,26.658705,Inhibitory,83.664066,71.656256,71.427088


In [178]:
cre.to_csv('cre_revised.csv', index = False)